In [38]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score,f1_score, confusion_matrix


In [39]:
from sklearn.metrics import accuracy_score

In [40]:
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score,StratifiedKFold

In [41]:
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier

In [42]:
from category_encoders import TargetEncoder

In [43]:
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor, Pool

In [44]:
import catboost

In [45]:
df = pd.read_csv("train.csv")

In [46]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [47]:
df.drop(columns=["PassengerId", "Name"],inplace=True )

In [48]:
df[["c1", "c2", "c3"]] = df["Cabin"].str.split("/", 2, expand=True)

C:\Users\User\AppData\Local\Temp\ipykernel_9712\2437142087.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[["c1", "c2", "c3"]] = df["Cabin"].str.split("/", 2, expand=True)


In [49]:
df.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,c1,c2,c3
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,B,0,P
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,F,0,S
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,A,0,S
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,A,0,S
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,F,1,S


In [50]:
df["c3"].nunique()

2

In [51]:
df["c2"].nunique()

1817

In [52]:
df["c1"].nunique()

8

In [53]:
df.drop(columns=["c2"], inplace=True)

In [54]:
df.drop(columns=["Cabin"], inplace=True)

In [55]:
df["Transported"] = df["Transported"] .map({True:1, False:0})

In [56]:
pop_id_cabin = df.pop("Transported")
df.insert(len(df.columns), 'Transported', pop_id_cabin)

In [57]:
df.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,c1,c3,Transported
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,B,P,0
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,F,S,1
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,A,S,0
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,A,S,0
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,F,S,1


In [58]:
X = df.iloc[:,:len(df.columns)-1]
y = df["Transported"]

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [60]:
cat = np.array([coluna for coluna in X_train.columns if X_train[coluna].dtypes.name=="object"])

In [61]:
cat

array(['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'c1', 'c3'],
      dtype='<U11')

In [62]:
num = np.array([coluna for coluna in X_train.columns if coluna not in cat])

In [63]:
num

array(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'],
      dtype='<U12')

In [64]:
te = TargetEncoder(
    cols=cat,
    smoothing=0.0,           # you can tweak this
    handle_unknown='value',  # how to treat unseen categories
    handle_missing='return_nan'  # <-- preserves NaNs instead of filling
)

# 3. Fit on your training data
#    Only fit on the categorical columns and the target
te.fit(X_train[cat], y_train)

TargetEncoder(cols=['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'c1',
                    'c3'],
              handle_missing='return_nan', smoothing=0.0)

In [65]:
X_train_enc = te.transform(X_train[cat])

In [66]:
X_test_2 = te.transform(X_test[cat])

In [67]:
X_test = pd.concat([
    X_test.drop(columns=cat).reset_index(drop=True),
    X_test_2.reset_index(drop=True)
], axis=1)

In [68]:
X_test.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet,CryoSleep,Destination,VIP,c1,c3
0,19.0,417.0,349.0,634.0,3.0,1057.0,0.527650,0.330327,0.474243,0.506838,0.430128,0.557219
1,18.0,4.0,904.0,0.0,0.0,1.0,0.422499,0.330327,0.474243,0.506838,0.518461,0.557219
2,41.0,0.0,0.0,0.0,0.0,0.0,0.422499,0.818828,0.474243,0.506838,0.518461,0.557219
3,35.0,0.0,338.0,436.0,NaN,0.0,0.422499,0.330327,0.474243,0.506838,0.518461,0.449083
4,43.0,0.0,0.0,0.0,0.0,0.0,0.658354,0.818828,0.474243,0.506838,0.433908,0.449083


In [69]:
X_train = pd.concat([
    X_train.drop(columns=cat).reset_index(drop=True),
    X_train_enc.reset_index(drop=True)
], axis=1)

In [70]:
X_train.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet,CryoSleep,Destination,VIP,c1,c3
0,27.0,441.0,0.0,397.0,471.0,0.0,0.527650,0.330327,0.474243,0.506838,0.430128,0.557219
1,45.0,0.0,0.0,0.0,0.0,0.0,0.658354,0.818828,0.601913,0.506838,0.726510,0.557219
2,50.0,0.0,0.0,0.0,0.0,0.0,0.658354,0.818828,0.474243,0.506838,0.726510,0.557219
3,1.0,0.0,0.0,0.0,0.0,0.0,0.422499,0.818828,0.474243,0.506838,0.518461,0.449083
4,42.0,0.0,29.0,317.0,434.0,45.0,0.422499,0.330327,0.474243,0.506838,0.518461,0.449083


In [71]:
# xg = xgb.XGBRegressor()

In [72]:
cr = xgb.XGBRegressor()

In [73]:
imputer = IterativeImputer(random_state=100, estimator=cr)

In [74]:
imputer.fit(X_train)


g:\Anaconda\Lib\site-packages\sklearn\impute\_iterative.py:796: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


IterativeImputer(estimator=XGBRegressor(base_score=None, booster=None,
                                        callbacks=None, colsample_bylevel=None,
                                        colsample_bynode=None,
                                        colsample_bytree=None,
                                        early_stopping_rounds=None,
                                        enable_categorical=False,
                                        eval_metric=None, feature_types=None,
                                        gamma=None, gpu_id=None,
                                        grow_policy=None, importance_type=None,
                                        interaction_constraints=None,
                                        learning_rate=None, max_bin=None,
                                        max_cat_threshold=None,
                                        max_cat_to_onehot=None,
                                        max_delta_step=None, max_depth=None,
                                        max_leaves=None, min_child_weight=None,
                                        missing=nan, monotone_constraints=None,
                                        n_estimators=100, n_jobs=None,
                                        num_parallel_tree=None, predictor=None,
                                        random_state=None, ...),
                 random_state=100)

In [75]:
imputed  = imputer.transform(X_train)

In [76]:
X_train_imputed = pd.DataFrame(
    imputed,
    columns=X_train.columns,
    index=X_train.index
)

In [77]:
X_train = X_train_imputed

In [78]:
models = []
models.append(("LGBMtun",LGBMClassifier(learning_rate=0.05, num_leaves=20, max_depth=14,verbose=-1)))
models.append(("xgb",xgb.XGBClassifier()))
models.append(("cat2",CatBoostClassifier(bagging_temperature=0.2938751257578318,border_count=170,depth=4,
                                        iterations=97,l2_leaf_reg=6,learning_rate=0.37852785960662555,
                                        random_strength=0.6918358593649996,scale_pos_weight=0.7881952275877159,
                                        verbose=False)))
models.append(("cat",CatBoostClassifier(verbose=False)))
models.append(("LGBM",LGBMClassifier(learning_rate=0.05, max_depth=20, n_estimators=200,verbose=-1)))
models.append(("xgb50,3",xgb.XGBClassifier(n_estimators=50, learning_rate=0.3)))
models.append(("LGBM2",LGBMClassifier(learning_rate=0.2,verbose=-1)))

results = dict()

for  name, model in models:
	skf = StratifiedKFold(n_splits = 5, random_state=None)
	cv_results = cross_val_score(model,X_train,y_train,cv=skf, scoring="accuracy")
	results[name]= (cv_results.mean(), cv_results.std())

print("name     results.mean     results.std")

for key,value in results.items():
	print(key,value)

name     results.mean     results.std
LGBMtun (0.8059528742072329, 0.004009165049885233)
xgb (0.8030369981496216, 0.006321048848008952)
cat2 (0.8068725310632848, 0.005632814292892528)
cat (0.8057986760142946, 0.00399924751462389)
LGBM (0.8082536054127096, 0.004744905581510439)
xgb50,3 (0.804878195198433, 0.003003287047298684)
LGBM2 (0.8019640847689852, 0.004859940609686921)


In [79]:
lgbm_params = {"n_estimators":[100,200,300], 
               "learning_rate":[0.01,0.05,0.1,0.3],
               "max_depth":[20,50,80,100]}

In [80]:
lgbmc = LGBMClassifier()


In [81]:
grid_search = GridSearchCV (estimator = lgbmc,
                            param_grid = lgbm_params,
                            n_jobs=-1,
                            cv = 5,
                            scoring="accuracy",
                           error_score='raise')

In [82]:
grid_result = grid_search.fit(X_train, y_train)


final_model = lgbmc.set_params(**grid_result.best_params_)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 3286, number of negative: 3233
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001308 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1697
[LightGBM] [Info] Number of data points in the train set: 6519, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504065 -> initscore=0.016261
[LightGBM] [Info] Start training from score 0.016261


In [83]:
final_model.fit(X_train, y_train)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 3286, number of negative: 3233
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005053 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1697
[LightGBM] [Info] Number of data points in the train set: 6519, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504065 -> initscore=0.016261
[LightGBM] [Info] Start training from score 0.016261


LGBMClassifier(max_depth=50)

In [84]:
imputed_test = imputer.transform(X_test)

In [85]:
X_test_imputed = pd.DataFrame(
    imputed_test,
    columns=X_test.columns,
    index=X_test.index)

In [86]:
X_test = X_test_imputed

In [87]:
y_pred = final_model.predict(X_test)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [88]:
accuracy = accuracy_score(y_pred, y_test)
accuracy

0.7980680772769089

In [89]:
precision = precision_score(y_pred, y_test)
accuracy = accuracy_score(y_pred, y_test)
recall = precision_score(y_pred, y_test)
f1 = f1_score(y_pred, y_test)

score = []
score.append(("precision", precision))
score.append(("accuracy",accuracy))
score.append(("recall",recall))
score.append(("f1",f1))

score= pd.DataFrame(score)
score.rename(columns={0: "Metric", 1:"Result"}, inplace=True)

In [90]:
display(score)


,Metric,Result
0,precision,0.819597
1,accuracy,0.798068
2,recall,0.819597
3,f1,0.803051


In [91]:
testecsv = pd.read_csv("test.csv")


In [92]:
testecsv[["c1","c2","c3"]] = testecsv["Cabin"].str.split("/", n=2, expand=True)


In [93]:
X_teste = testecsv.drop(columns=["PassengerId","Cabin","Name"])



In [94]:
X_teste.drop(columns=["c2"], inplace=True)

In [95]:
X_teste.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,c1,c3
0,Earth,True,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,G,S
1,Earth,False,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,F,S
2,Europa,True,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,C,S
3,Europa,False,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,C,S
4,Earth,False,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,F,S


In [96]:
X_teste_enc = te.transform(X_teste[cat])

In [97]:
X_teste = pd.concat([
    X_teste.drop(columns=cat).reset_index(drop=True),
    X_teste_enc.reset_index(drop=True)
], axis=1)

In [98]:
imputedf = imputer.transform(X_teste)

In [99]:
X_teste = pd.DataFrame(
    imputedf,
    columns=X_teste.columns,
    index=X_teste.index
)

In [100]:
y_train

5623    0
5253    1
478     1
1352    1
5344    1
       ..
5734    1
5191    0
5390    0
860     0
7270    0
Name: Transported, Length: 6519, dtype: int64

In [101]:
X_all = pd.concat([X_train, X_test], axis=0).reset_index(drop=True)
y_all = pd.concat([y_train, y_test], axis=0).reset_index(drop=True)

In [102]:
final_model.fit(X_all, y_all)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 4378, number of negative: 4315
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011087 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1795
[LightGBM] [Info] Number of data points in the train set: 8693, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503624 -> initscore=0.014495
[LightGBM] [Info] Start training from score 0.014495


LGBMClassifier(max_depth=50)

In [103]:
y_predz = final_model.predict(X_teste)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [104]:
subimisspace = pd.Series(index = testecsv["PassengerId"].values, data = y_predz)


In [105]:
subimisspace = subimisspace.reset_index()


In [106]:
subimisspace = pd.DataFrame(subimisspace)


In [107]:
subimisspace[0]=subimisspace[0].map({1:"True", 0:"False"})


In [108]:
subimisspace.rename(columns = {"index":"PassengerId", 0:"Transported"}, inplace=True)


In [109]:
subimisspace.to_csv("lajoy2.csv", index=False)
